In [7]:
import os, glob
import pandas as pd
from io import StringIO
import numpy as np
import pysam
import pybedtools 
from Bio import SeqIO
from Bio.Seq import Seq
pybedtools.helpers.set_tempdir("/data/projects/temp")

In [8]:
non_coding_region = "core_prom"

In [9]:
data_path = '/data/projects/DNABERT_snv/Manuscript_11_2023/Intersected_data/{}_ICGC_intersected.tsv'.format(non_coding_region)
output_path = "/data/projects/DNABERT_snv/Manuscript_11_2023/DNABERT_data/{}".format(non_coding_region) 

In [10]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [11]:
def seq2kmer(seq):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space
    """
    k=6
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [12]:
genome = SeqIO.to_dict(SeqIO.parse("/data/projects/Resources/Gencode_genome_annotation/GRCh38.primary_assembly.genome.fa", "fasta"))

In [13]:
df = pd.read_csv(data_path, sep= '\t')
print(df.shape)

(746364, 22)


In [14]:
# df = pd.read_csv(data_path, sep= '\t')
# print(df.shape)

# df = df[df['No_of_variations']<3].reset_index(drop=True)
# print(df.shape)
# df = df.loc[df['Alternative_Allele']== '.'].reset_index(drop=True)
# df
#df= df[df['strand']=="-"]


data= []
#df = df.head(10)
# Iterate over DataFrame rows
for idx, row in df.iterrows():
    try:
        #print(row)Reference_Allele	Alternative_Allele
        alts = row['Alternative_Allele'].rstrip(',').split(',')
        ref_seq = str(genome[row['chr_name']].seq[ row['Core_prom_start']:row['Core_prom_end']])

        # Calculate the variant position relative to the fetched sequence
        variant_pos_start = row['Start'] - row['Core_prom_start']
        variant_pos_end = row['End']  - row['Core_prom_start']
        # print(variant_pos_start, variant_pos_end, alts,  row['Reference_Allele'])
        # print(ref_seq)
        # if row['strand'] == '-':
        #     ref_seq = str(Seq(ref_seq).reverse_complement())
        #print(ref_seq)


        #Replace the reference nucleotide with the alternate nucleotide to get the alternate sequence
        for alt in alts:
            if alt == ".":  # Deletion
                # Fetch additional bases from genome to maintain sequence length
                extra_bases = str(genome[row['chr_name']].seq[row['Core_prom_end']:row['Core_prom_end'] + len(row['Reference_Allele'])])
                #print(len(row['Reference_Nucleotide']))
                alt_seq = ref_seq[:variant_pos_start] + ref_seq[variant_pos_end:] + extra_bases
            else:  # SNPs, Insertions, and partial Deletions
                extra_bases = str(genome[row['chr_name']].seq[row['Core_prom_end']:row['Core_prom_end'] + len(row['Reference_Allele']) - len(alt)])
                #print(extra_bases)
                if(variant_pos_start>=0):
                    alt_seq = ref_seq[:variant_pos_start] + alt + ref_seq[variant_pos_end:] + extra_bases
                else:
                    continue
            #print(alt_seq)
            #Append to the list as a dictionary
            data.append({
                'chr': row['chr_name'],
                'strand': row['strand'],
                'Core_prom_coordinates': str(row['Core_prom_start'])+'-'+str(row['Core_prom_end']),
                'Ensemble_Transcript_ID': row['ENSEMBL_Transcript_ID'],
                'Mutation_ID': row['Mutation_ID'],
                'varinat_start':  row['Start'],
                'variant_end':  row['End'],
                'ref_neucleotide': row['Reference_Allele'],
                'alternative_neucleotide': alt,
                'reference_seq': ref_seq,
                'alt_seq': alt_seq
            })
            
    except KeyError:
        print("NA")
        # # If sequence is not present, append a default value
        # ref_sequences.append("NA")
        # alt_sequences.append("NA")

        
        
new_df = pd.DataFrame(data)
print(new_df.shape)
new_df = new_df.drop_duplicates().reset_index(drop=True)
print(new_df.shape)
merged_list = list(zip(new_df['reference_seq'], new_df['alt_seq']))
merged_list = [item.upper() for tup in merged_list for item in tup]
#print(merged_list)
kmer_lst = list(map(seq2kmer, merged_list))
df_kmer = pd.DataFrame(kmer_lst, columns=['Sequence'])
df_kmer['Label'] = np.random.choice([0, 1], size=len(df_kmer))

new_df.to_csv(output_path + "/all_data_new.tsv", sep="\t", index= False)
df_kmer.to_csv(output_path + "/dev.tsv", sep="\t", index= False)

(745265, 11)
(745265, 11)


In [15]:
new_df

,chr,strand,Core_prom_coordinates,Ensemble_Transcript_ID,Mutation_ID,varinat_start,variant_end,ref_neucleotide,alternative_neucleotide,reference_seq,alt_seq
0,chr7,+,127588366-127588455,ENST00000000233,MU133392649,127588387,127588388,G,A,GTCGGGAGGGCAGCGACGCGCGGAGGCGGCGGCGGAGCCTCCTCCT...,GTCGGGAGGGCAGCGACGCGCAGAGGCGGCGGCGGAGCCTCCTCCT...
1,chr7,+,127588366-127588455,ENST00000000233,MU39478069,127588429,127588430,T,C,GTCGGGAGGGCAGCGACGCGCGGAGGCGGCGGCGGAGCCTCCTCCT...,GTCGGGAGGGCAGCGACGCGCGGAGGCGGCGGCGGAGCCTCCTCCT...
2,chr12,-,8949601-8949690,ENST00000000412,MU49180650,8949659,8949660,G,A,TCAGAGGCCAGCGTTCCCCCTAGCGCCTCGCTGTGCCCCACTCTGG...,TCAGAGGCCAGCGTTCCCCCTAGCGCCTCGCTGTGCCCCACTCTGG...
3,chr12,-,8949601-8949690,ENST00000000412,MU37117518,8949660,8949661,G,A,TCAGAGGCCAGCGTTCCCCCTAGCGCCTCGCTGTGCCCCACTCTGG...,TCAGAGGCCAGCGTTCCCCCTAGCGCCTCGCTGTGCCCCACTCTGG...
4,chr11,+,64305479-64305568,ENST00000000442,MU46907284,64305485,64305486,C,T,CCCGCCCCCTGCTTTGCATGCGCACGGCCGGCCCCACCCCCGCTGT...,CCCGCCTCCTGCTTTGCATGCGCACGGCCGGCCCCACCCCCGCTGT...
...,...,...,...,...,...,...,...,...,...,...,...
745260,chr20,+,45416088-45416177,ENST00000640996,MU66504628,45416128,45416129,C,G,CCTTTCTTATTGGATACCGGCTTTTCATGGGGCCGAGCGCCGCTGG...,CCTTTCTTATTGGATACCGGCTTTTCATGGGGCCGAGCGCGGCTGG...
745261,chr20,+,45416088-45416177,ENST00000640996,MU128987750,45416129,45416130,G,A,CCTTTCTTATTGGATACCGGCTTTTCATGGGGCCGAGCGCCGCTGG...,CCTTTCTTATTGGATACCGGCTTTTCATGGGGCCGAGCGCCACTGG...
745262,chr20,+,45416088-45416177,ENST00000640996,MU120160683,45416141,45416142,G,A,CCTTTCTTATTGGATACCGGCTTTTCATGGGGCCGAGCGCCGCTGG...,CCTTTCTTATTGGATACCGGCTTTTCATGGGGCCGAGCGCCGCTGG...
745263,chr20,+,45416088-45416177,ENST00000640996,MU93024897,45416144,45416145,G,T,CCTTTCTTATTGGATACCGGCTTTTCATGGGGCCGAGCGCCGCTGG...,CCTTTCTTATTGGATACCGGCTTTTCATGGGGCCGAGCGCCGCTGG...


In [16]:
df_kmer

,Sequence,Label
0,GTCGGG TCGGGA CGGGAG GGGAGG GGAGGG GAGGGC AGGG...,0
1,GTCGGG TCGGGA CGGGAG GGGAGG GGAGGG GAGGGC AGGG...,0
2,GTCGGG TCGGGA CGGGAG GGGAGG GGAGGG GAGGGC AGGG...,1
3,GTCGGG TCGGGA CGGGAG GGGAGG GGAGGG GAGGGC AGGG...,1
4,TCAGAG CAGAGG AGAGGC GAGGCC AGGCCA GGCCAG GCCA...,1
...,...,...
1490525,CCTTTC CTTTCT TTTCTT TTCTTA TCTTAT CTTATT TTAT...,0
1490526,CCTTTC CTTTCT TTTCTT TTCTTA TCTTAT CTTATT TTAT...,1
1490527,CCTTTC CTTTCT TTTCTT TTCTTA TCTTAT CTTATT TTAT...,1
1490528,CCTTTC CTTTCT TTTCTT TTCTTA TCTTAT CTTATT TTAT...,0
